In [5]:
%load_ext autoreload
%autoreload 2

from chess import Game
from chess.pieces import *
import random
import time
import cProfile

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import numpy as np

class Agent:
    
    def __init__(self):
        self.piece_value = {
            'Empty': 0,
            'Pawn': 1,
            'Horse': 3,
            'Bishop': 3,
            'Tower': 5,
            'Queen': 9,
            'King': 1e6,
        }
        
    def state_value(self, game):
        value = 0
        for row in game.board:
            for piece in row:
                piece_value = self.piece_value[type(piece).__name__]
                if piece.color == 'black':
                    value += piece_value
                else:
                    value -= piece_value
        return value
    
    def policy(self, game):
        max_value = - np.inf
        for move in game.moves('black'):
            root_node = game.simulate_move(*move)
            value = self.alphabeta(root_node, 3, False)
            if value >= max_value:
                best_move = move
                self._value = value
        return best_move

    def alphabeta(self, node, depth, alpha=-np.inf, beta=np.inf, maximizing_player=True):
        moves = node.moves(node.current_color)
        if depth == 0 or len(moves) == 0:
            return self.state_value(node)
        if maximizing_player:
            value = - np.inf
            for move in moves:
                new_node = node.simulate_move(*move)
                value = max(value, self.alphabeta(new_node, depth - 1, alpha, beta, False))
                alpha = max(alpha, value)
                if alpha >= beta:
                    break
            return value
        else:
            value = np.inf
            for move in moves:
                new_node = node.simulate_move(*move)
                value = min(value, self.alphabeta(new_node, depth - 1, alpha, beta, True))
                beta = min(beta, value)
                if alpha >= beta:
                    break
            return value

In [7]:
import cProfile

game = Game()
agent = Agent()

cProfile.run('agent.alphabeta(game, 4)')

         7998970 function calls (7164946 primitive calls) in 4.997 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    53688    0.052    0.000    0.191    0.000 <__array_function__ internals>:2(where)
      893    0.030    0.000    0.032    0.000 <ipython-input-6-4df8651bae9f>:16(state_value)
   1417/1    0.012    0.000    4.997    4.997 <ipython-input-6-4df8651bae9f>:37(alphabeta)
        1    0.000    0.000    4.997    4.997 <string>:1(<module>)
   359084    0.051    0.000    0.051    0.000 board.py:20(board)
   332776    0.353    0.000    0.713    0.000 board.py:45(__getitem__)
   974417    0.142    0.000    0.142    0.000 board.py:46(<genexpr>)
     2832    0.004    0.000    0.004    0.000 board.py:53(__setitem__)
    43905    0.006    0.000    0.006    0.000 board.py:57(__len__)
    53688    0.620    0.000    0.639    0.000 board.py:60(__eq__)
     2310    0.003    0.000    0.003    0.000 board.py:67(__iter__)
649944/1

In [22]:
game = Game()
agent = Agent()


while not (game.is_checkmate(game.current_color) or game.is_draw()):
    print(game.current_color)
    if game.current_color == 'white':
        from_, to = random.choice(game.moves(game.current_color))
        new_game = game.simulate_move(from_, to)
        if new_game.is_check('white'):
            continue
    else:
        from_, to = agent.policy(game)
        print(f'agent move, value: {agent._value}..')
    game.move(from_, to)
    print(f"{from_} -> {to}")
    print(game)

black
agent move, value: 0.0..
(1, 7) -> (3, 7)
8 ♜ ♞ ♝ ♚ ♛ ♝ ♞ ♜
7 ♟ ♟ ♟ ♟ ♟ ♟ ♟  
6                
5               ♟
4                
3                
2 ♙ ♙ ♙ ♙ ♙ ♙ ♙ ♙
1 ♖ ♘ ♗ ♔ ♕ ♗ ♘ ♖
  a b c d e f g h
white
(7, 6) -> (5, 7)
8 ♜ ♞ ♝ ♚ ♛ ♝ ♞ ♜
7 ♟ ♟ ♟ ♟ ♟ ♟ ♟  
6                
5               ♟
4                
3               ♘
2 ♙ ♙ ♙ ♙ ♙ ♙ ♙ ♙
1 ♖ ♘ ♗ ♔ ♕ ♗   ♖
  a b c d e f g h
black
agent move, value: 0.0..
(3, 7) -> (4, 7)
8 ♜ ♞ ♝ ♚ ♛ ♝ ♞ ♜
7 ♟ ♟ ♟ ♟ ♟ ♟ ♟  
6                
5                
4               ♟
3               ♘
2 ♙ ♙ ♙ ♙ ♙ ♙ ♙ ♙
1 ♖ ♘ ♗ ♔ ♕ ♗   ♖
  a b c d e f g h
white
(7, 1) -> (5, 0)
8 ♜ ♞ ♝ ♚ ♛ ♝ ♞ ♜
7 ♟ ♟ ♟ ♟ ♟ ♟ ♟  
6                
5                
4               ♟
3 ♘             ♘
2 ♙ ♙ ♙ ♙ ♙ ♙ ♙ ♙
1 ♖   ♗ ♔ ♕ ♗   ♖
  a b c d e f g h
black
agent move, value: 0.0..
(1, 6) -> (3, 6)
8 ♜ ♞ ♝ ♚ ♛ ♝ ♞ ♜
7 ♟ ♟ ♟ ♟ ♟ ♟    
6                
5             ♟  
4               ♟
3 ♘             ♘
2 ♙ ♙ ♙ ♙ ♙ ♙ ♙ ♙
1 ♖   ♗ ♔ ♕ ♗   ♖
  a b c d e f g h


SystemError: <built-in method __deepcopy__ of numpy.ndarray object at 0x7f58347bb3f0> returned a result with an error set